In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM

In [2]:
data=pd.read_csv("C:/Users/absub/Downloads/electricity_data1.csv")
data.head()

,Timestamp,Power Load (MW),Temperature (°C),Humidity (%),Wind Speed (km/h),Rain (mm),Public Holiday (0/1),Weekday/Weekend (0/1),Solar Generation (MW),Region,Population Growth Rate (%)
0,2015-01-01 00:00:00,918.373818,38.253605,89.370481,0.280318,8.062136,1,1,353.567576,South,0.045795
1,2015-01-01 01:00:00,1843.312844,15.214944,82.738678,6.643337,2.924700,0,0,365.597743,East,0.030522
2,2015-01-01 02:00:00,1180.126923,15.070492,20.579996,10.830845,9.643237,0,0,276.744449,East,0.041935
3,2015-01-01 03:00:00,1951.315903,29.744690,69.338933,20.624899,4.982557,1,1,155.461630,East,0.049135
4,2015-01-01 04:00:00,614.501023,28.058788,33.189133,11.544552,3.864432,1,0,349.055289,West,0.023726


In [3]:
data.shape

(85417, 11)

In [4]:
# Feature Engineering (assuming the dataset has timestamp and other features)
data['hour'] = pd.to_datetime(data['Timestamp']).dt.hour
data['day_of_week'] = pd.to_datetime(data['Timestamp']).dt.dayofweek
data['month'] = pd.to_datetime(data['Timestamp']).dt.month
data.head()

,Timestamp,Power Load (MW),Temperature (°C),Humidity (%),Wind Speed (km/h),Rain (mm),Public Holiday (0/1),Weekday/Weekend (0/1),Solar Generation (MW),Region,Population Growth Rate (%),hour,day_of_week,month
0,2015-01-01 00:00:00,918.373818,38.253605,89.370481,0.280318,8.062136,1,1,353.567576,South,0.045795,0,3,1
1,2015-01-01 01:00:00,1843.312844,15.214944,82.738678,6.643337,2.924700,0,0,365.597743,East,0.030522,1,3,1
2,2015-01-01 02:00:00,1180.126923,15.070492,20.579996,10.830845,9.643237,0,0,276.744449,East,0.041935,2,3,1
3,2015-01-01 03:00:00,1951.315903,29.744690,69.338933,20.624899,4.982557,1,1,155.461630,East,0.049135,3,3,1
4,2015-01-01 04:00:00,614.501023,28.058788,33.189133,11.544552,3.864432,1,0,349.055289,West,0.023726,4,3,1


In [5]:
# Select features (including Region) and target
X = data[['Temperature (°C)','Humidity (%)','Wind Speed (km/h)','Rain (mm)','Public Holiday (0/1)','Weekday/Weekend (0/1)',
          'Solar Generation (MW)','Region','Population Growth Rate (%)','hour','day_of_week','month']]
y = data['Power Load (MW)']

In [6]:
print(X.shape)
print(y.shape)

(85417, 12)
(85417,)


In [7]:
# Example of label encoding for 'Region' to avoid expanding columns
label_encoder = LabelEncoder()
data['Region'] = label_encoder.fit_transform(data['Region'])
#print(data['Region'])

In [8]:
# List of numerical features to be scaled
numerical_features = ['Temperature (°C)', 'Humidity (%)', 'Wind Speed (km/h)', 
                      'Rain (mm)', 'Public Holiday (0/1)', 'Weekday/Weekend (0/1)',
                      'Solar Generation (MW)', 'Population Growth Rate (%)', 
                      'hour', 'day_of_week', 'month']

In [9]:
# Initialize the scaler
scaler = MinMaxScaler()

In [10]:
# Apply scaling to the numerical features
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [11]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Apply label encoding to the 'Region' column
X_train['Region'] = label_encoder.fit_transform(X_train['Region'])
X_test['Region'] = label_encoder.transform(X_test['Region'])

LSTM-->Long Short Term Memory

In [13]:
# Convert DataFrame to NumPy array
X_train = X_train.values
X_test= X_test.values

In [14]:
# Reshape to 3D: (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [15]:
print(X_train.shape) 

(68333, 1, 12)


In [16]:
# Define LSTM model
lstm_model = Sequential()

In [17]:
# Add LSTM layers
lstm_model.add(LSTM(100, activation='relu',input_shape=(X_train.shape[1], X_train.shape[2])))
lstm_model.add(Dense(50, activation='relu'))
lstm_model.add(Dense(1))

C:\Users\absub\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
# Train the model
lstm_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 486654.0938 - val_loss: 205964.4844
Epoch 2/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 206478.6875 - val_loss: 194800.4688
Epoch 3/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 196297.8594 - val_loss: 190117.3750
Epoch 4/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 192329.0625 - val_loss: 188833.0781
Epoch 5/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 192585.6719 - val_loss: 188657.1406
Epoch 6/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 193374.6094 - val_loss: 196744.5312
Epoch 7/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 193663.4375 - val_loss: 189406.8438
Epoch 8/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 191938.5156 - val_loss: 188251.4844
Epoch 9/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 192745.5000 - val_loss: 190550.6094
Epoch 10/50
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 191732.4062 - val_loss: 189899.8438
Epoch 11/50
1068/1

In [20]:
# Predict and evaluate
lstm_y_pred = lstm_model.predict(X_test)
lstm_rmse = np.sqrt(np.mean((y_test - lstm_y_pred.reshape(-1))**2))
print(f'LSTM RMSE: {lstm_rmse}')

534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
LSTM RMSE: 433.3481878389035


In [21]:
# Example input factors (reshaped to match LSTM's input requirements)
new_data =np.array([[
    40, # Temperature
    65,  # Humidity
    15,  # Wind Speed
    0,   # Rain (mm)
    0,   # Public Holiday (0 or 1)
    1,   # Weekday/Weekend (0 or 1)
    120, # Solar Generation (MW)
    2,    # Region (encoded)
    1.2, # Population Growth Rate
    12,  # Hour of the day
    2,   # Day of the week
    9   # Month
]])

In [22]:
# Reshape to (samples, time_steps, features)
new_data = new_data.reshape((1, 1, new_data.shape[1]))  # (1 sample, 1 time step, 12 features)

In [23]:
predicted_power_load = lstm_model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step


In [24]:
print(f"Predicted Power Load (MW): {predicted_power_load[0][0]}")

Predicted Power Load (MW): 1447.464599609375
